In [ ]:
from tensorflow.keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# first we fit the scaler on the training dataset
scaler.fit(train_data)

# then we call the transform method to scale both the training and testing data
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers

def build_model(number_of_hidden_layers=1, number_of_neurons=2):
    model = Sequential()

    # First hidden layer
    model.add(Dense(number_of_neurons, activation='relu', input_shape=(train_data.shape[1],)))

    # hidden layers
    for hidden_layer_number in range(1, number_of_hidden_layers):
        model.add(Dense(number_of_neurons, activation='relu'))

    # output layer
    model.add(Dense(1))

    model.compile(optimizer='adam',loss='mse',metrics=['mae',])
    return model

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor # There is also a KerasClassifier class
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, KFold

In [ ]:
tuned_model = KerasRegressor(build_fn=build_model)

# possible values of parameters - we want to find the best set of them
# maybe it is best to create a model with 3 hidden layers, with 15 neurons in each hidden layer?
# And maybe it would be best to create a model with 5 hidden layers, with 5 neurons in each hidden layer? 
# Let's find out!
params = {
      'number_of_hidden_layers': [1, 2, 3, 4, 5, 6],
      'number_of_neurons': [8, 16, 32, 64]
      }

# Create a randomize search cross validation object, to find the best hyperparameters it will use a KFold cross validation with 4 splits
random_search = RandomizedSearchCV(tuned_model, param_distributions = params, cv = KFold(4))

# Note: RandomizedSearchCV does not check all combinations of parameters - it picks the combinations of hyperparameters to check randomly.
# To check all possible combinations of hyperparameters, use GridSearchCV. (Computations will last longer!)

# find the best parameters!
random_search.fit(train_data, train_targets)

In [ ]:
random_search.best_estimator_.get_params()
# It was found that the best combination of hyperparameters is:
  #  'number_of_neurons': 64
  #  'number_of_hidden_layers': 6

In [ ]:
grid_search = GridSearchCV(tuned_model, param_grid = params, cv = KFold(4))

# Note: RandomizedSearchCV does not check all combinations of parameters - it picks the combinations of hyperparameters to check randomly.
# To check all possible combinations of hyperparameters, use GridSearchCV. (Computations will last longer!)

# find the best parameters!
grid_search.fit(train_data, train_targets)

In [ ]:
grid_search.best_estimator_.get_params()
# It was found that the best combination of hyperparameters is:
  #  'number_of_neurons': 64
  #  'number_of_hidden_layers': 3

In [ ]:
best_found_model1 = build_model(6, 64)
best_found_model2 = build_model(3,64)

In [ ]:
history1 = best_found_model1.fit(train_data, train_targets, epochs=80)

In [ ]:
history2 = best_found_model2.fit(train_data, train_targets, epochs=80)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, (len(history1.history['mae']) + 1)), history1.history['mae'])
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, (len(history2.history['mae']) + 1)), history2.history['mae'])
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.show()

In [ ]:
test_mse_score, test_mae_score = best_found_model1.evaluate(test_data, test_targets)

In [ ]:
test_mse_score, test_mae_score = best_found_model2.evaluate(test_data, test_targets)

In [ ]:
to_predict = test_data[[1, 25, 50, 75, 100]]
predictions = best_found_model1.predict(to_predict)
print(predictions)

print(test_targets[[1, 25, 50, 75, 100]])

In [ ]:
to_predict = test_data[[1, 25, 50, 75, 100]]
predictions = best_found_model2.predict(to_predict)
print(predictions)

print(test_targets[[1, 25, 50, 75, 100]])